In [ ]:
import pandas as pd
import numpy as np
import logit

In [ ]:
dataset = pd.read_csv('swissmetro.dat', sep='\t').query('(PURPOSE == 1 or PURPOSE == 3) and CHOICE != 0')
dataset.head()

In [ ]:
# This will be the initial starting point.
betas = logit.Betas(asc_car=0, asc_train=0, asc_sm=0, time=0, cost=0)

utilities = {
    # train
    1: lambda b, d: betas.get('asc_train', b) +
                        betas.get('time', b) * d['TRAIN_TT'] +
                        betas.get('cost', b) * d['TRAIN_CO'],
    # SwissMetro
    2: lambda b, d: betas.get('asc_sm', b) +
                        betas.get('time', b) * d['SM_TT'] +
                        betas.get('cost', b) * d['SM_CO'],
    # Car
    3: lambda b, d: betas.get('asc_car', b) +
                        betas.get('time', b) * d['CAR_TT'] +
                        betas.get('cost', b) * d['CAR_CO']
}


In [ ]:
estimates = logit.estimate_logit(betas, utilities, dataset.CHOICE, dataset)

In [ ]:
estimates